# Gestion d'une api

## Routes

In [5]:
from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
from typing import List

router = APIRouter()

class Item(BaseModel):
    id: str
    name: str
    description: str

items_db = []

@router.post("/items/", response_model=Item)
def create_item(item: Item):
    items_db.append(item)
    return item

@router.get("/items/", response_model=List[Item])
def read_items():
    return items_db

@router.get("/items/{item_id}", response_model=Item)
def read_item(item_id: str):
    for item in items_db:
        if item.id == item_id:
            return item
    raise HTTPException(status_code=404, detail="Item not found")

## Models

In [6]:
from pydantic import BaseModel
from typing import List, Optional

class Repo(BaseModel):
    name: str
    full_name: str
    html_url: str
    description: Optional[str] = None
    readme: str

## Dependences

In [7]:
from motor.motor_asyncio import AsyncIOMotorClient

def get_db_connection():
    client = AsyncIOMotorClient('mongodb://mongo:27017')
    return client.bdProjet

def get_elasticsearch_client():
    # Logic to get an Elasticsearch client
    pass

## Routes

In [8]:
from fastapi import FastAPI, APIRouter, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from typing import List

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(router)

@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI application!"}



router = APIRouter()

@router.get("/repos/", response_model=List[Repo])
async def read_repos(db=Depends(get_db_connection)):
    repos = await db.repos.find().to_list(100)
    return repos

@router.get("/repos/{repo_id}", response_model=Repo)
async def read_repo(repo_id: str, db=Depends(get_db_connection)):
    repo = await db.repos.find_one({"_id": repo_id})
    if repo is None:
        raise HTTPException(status_code=404, detail="Repo not found")
    return repo